In [1]:
import kfp
import kfp.dsl as dsl
import kfp.components as comp

In [2]:
client = kfp.Client()

ERROR:root:Failed to read a token from file '/var/run/secrets/kubeflow/pipelines/token' ([Errno 2] No such file or directory: '/var/run/secrets/kubeflow/pipelines/token').


In [66]:
create_step_get_lines = comp.load_component_from_text("""
name: Get Lines
description: Gets the specified number of lines from the input file.

inputs:
- {name: file_path, type: text, description: 'Input file path'}

implementation:
  container:
    image: debian:latest
    command: [
      cat,      
      {inputPath: file_path},
    ]""")

In [147]:
def add(
    a: float, 
    b: float,
    output_text: comp.OutputPath('text'),
):
    import json
    
    a = float(a)
    b = float(b)    
    for i in range(10):
        print(f'multiply={a * b}')
        
    with open(output_text, 'w') as f:
        for i in range(10):
            f.write(f'multiply={a * b}\n')

component_add = kfp.components.create_component_from_func(
    func=add,
    base_image='python:3.7',
    packages_to_install=[],
)
    
def my_trivial_pipeline(a, b):
    add_task = component_add(a, b)
    create_step_get_lines(
        file_path=add_task.outputs['output_text'],
    )

In [148]:
kfp.compiler.Compiler().compile(
    pipeline_func=my_trivial_pipeline,
    package_path='trivial.yaml'
)

In [149]:
client.create_run_from_pipeline_func(
    my_trivial_pipeline,
    arguments={
        'a': 50.,
        'b': -2.,
    },
    experiment_name='pavlos-exp-1',
)

RunPipelineResult(run_id=1b77ccb0-c4ef-4263-b7cf-4df14b324eaf)

In [150]:
import kfp
import kfp.dsl as dsl
from kfp import components

from kubeflow.katib import ApiClient
from kubernetes.client import V1ObjectMeta
from kubeflow.katib import V1beta1Experiment as Experiment
from kubeflow.katib import V1beta1ExperimentSpec as ExperimentSpec
from kubeflow.katib import V1beta1AlgorithmSpec as Algorithm
from kubeflow.katib import V1beta1MetricsCollectorSpec as MetricsCollector
from kubeflow.katib import V1beta1CollectorSpec as Collector
from kubeflow.katib import V1beta1SourceSpec as Source
from kubeflow.katib import V1beta1FileSystemPath as FileSysPath
from kubeflow.katib import V1beta1FilterSpec as Filter
from kubeflow.katib import V1beta1EarlyStoppingSpec as EarlyStopping
from kubeflow.katib import V1beta1EarlyStoppingSetting as EarlyStoppingSetting
from kubeflow.katib import V1beta1ObjectiveSpec as Objective
from kubeflow.katib import V1beta1ParameterSpec as Parameter
from kubeflow.katib import V1beta1FeasibleSpace as FeasibleSpace
from kubeflow.katib import V1beta1TrialTemplate as TrialTemplate
from kubeflow.katib import V1beta1TrialParameterSpec as TrialParameter
from kubeflow import katib

In [151]:
compiler = kfp.compiler.Compiler()

In [152]:
trail_spec = compiler._create_workflow(
    pipeline_func=my_trivial_pipeline,
    params_list=[
        kfp.dsl._pipeline_param.PipelineParam(
            name='a', 
            value='${trialParameters.a}',
        ),

        kfp.dsl._pipeline_param.PipelineParam(
            name='b', 
            value='${trialParameters.b}',
        ),                

    ],
)

In [153]:
trail_spec['spec']['serviceAccountName'] = 'default-editor'
trail_spec['metadata']['annotations']['sidecar.istio.io/inject'] = 'false'
trail_spec['metadata']['labels']['access-ml-pipeline'] = 'true'
for tmpl in trail_spec['spec']['templates']:
    if not 'metadata' in tmpl:
        continue
    tmpl['metadata']['annotations']['sidecar.istio.io/inject'] = 'false'
    tmpl['metadata']['labels']['access-ml-pipeline'] = 'true'    
    # tmpl['metadata']['labels']['katib.kubeflow.org/model-training'] = 'true'
    
    
(
    trail_spec['spec']['templates'][1]
    ['metadata']['labels']['katib.kubeflow.org/model-training']
) = 'true'    

In [154]:
experiment_spec = ExperimentSpec(
    max_trial_count=20,
    max_failed_trial_count=3,
    parallel_trial_count=2,
    objective=Objective(
        type="maximize",
        objective_metric_name='multiply',
    ),
    algorithm=Algorithm(algorithm_name="random"),
    parameters=[
        Parameter(
            name="a",
            parameter_type="double",
            feasible_space=FeasibleSpace(
                min="-5",
                max="2",
            ),
        ),
        Parameter(
            name="b",
            parameter_type="double",
            feasible_space=FeasibleSpace(
                min="-10.0",
                max="1.0",
            ),
        ),        
    ],
    trial_template=TrialTemplate(
        retain=True,
        primary_container_name="main",
        primary_pod_labels={
            "katib.kubeflow.org/model-training": "true",
        },
        success_condition = 'status.[@this].#(phase=="Succeeded")#',
        failure_condition = 'status.[@this].#(phase=="Failed")#',
        trial_parameters=[
            TrialParameter(
                name="a",
                description="a",
                reference="a",
            ),
            TrialParameter(
                name="b",
                description="b",
                reference="b",
            ),            
        ],
        trial_spec=trail_spec,
    ),
)

In [158]:
exp = Experiment(
    api_version="kubeflow.org/v1beta1",
    kind="Experiment",
    metadata=V1ObjectMeta(
        name="my-trivial-katib-v20",
        namespace="kubeflow-user-example-com",
    ),
    spec=experiment_spec,
)

In [ ]:
from kubeflow.katib import KatibClient
KatibClient().create_experiment(exp, namespace="kubeflow-user-example-com")